In [1]:
import pandas as pd
import os

# Set working directory
os.chdir(os.path.expanduser('~/Desktop/Media Bias Project'))
print(os.getcwd())

/Users/yunlin/Dropbox/My Mac (Yunlin’s MacBook Pro)/Desktop/Media Bias Project


In [2]:
# Load the dataset
df = pd.read_excel("data/labeled_dataset.xlsx")

/opt/anaconda3/envs/erdos_summer_2025/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [3]:
#  Drop the Unnamed: 0 column if present (index column)
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])

df.head()

,sentence,news_link,outlet,topic,type,group_id,num_sent,Label_bias,Label_opinion,article,biased_words4
0,YouTube is making clear there will be no “birt...,https://eu.usatoday.com/story/tech/2020/02/03/...,usa-today,elections-2020,center,1,1,Biased,Somewhat factual but also opinionated,YouTube says no ‘deepfakes’ or ‘birther’ video...,"['belated', 'birtherism']"
1,The increasingly bitter dispute between Americ...,https://www.nbcnews.com/news/sports/women-s-te...,msnbc,sport,left,1,1,Non-biased,Entirely factual,"FRISCO, Texas — The increasingly bitter disput...",['bitter']
2,So while there may be a humanitarian crisis dr...,https://www.alternet.org/2019/01/here-are-5-of...,alternet,immigration,left,1,1,Biased,Expresses writer’s opinion,Speaking to the country for the first time fro...,['crisis']
3,A professor who teaches climate change classes...,https://www.breitbart.com/politics/2019/05/09/...,breitbart,environment,right,1,1,Non-biased,No agreement,A professor who teaches climate change classes...,['legitimate']
4,"Looking around the United States, there is nev...",https://thefederalist.com/2020/03/11/woman-who...,federalist,abortion,right,1,1,Biased,Somewhat factual but also opinionated,The left has a thing for taking babies hostage...,"['killing', 'never', 'developing', 'humans', '..."


In [4]:
# Check missing values
print("Missing values per column:\n", df.isnull().sum())

Missing values per column:
 sentence           0
news_link         19
outlet             0
topic              0
type               0
group_id           0
num_sent           0
Label_bias         0
Label_opinion      0
article          100
biased_words4      0
dtype: int64


In [5]:
# Tabulate frequency of each news outlet in the dataset
outlet_counts = df['outlet'].value_counts()
print("Outlet counts:\n", outlet_counts)

# Tabulate frequency of each news outlet type
type_counts = df['type'].value_counts()
print("Outlet type counts:\n", type_counts)

Outlet counts:
 outlet
breitbart     377
alternet      338
msnbc         242
federalist    203
reuters       158
usa-today     157
huffpost      114
fox-news      111
Name: count, dtype: int64
Outlet type counts:
 type
left      694
right     691
center    315
Name: count, dtype: int64


In [6]:
# Tabulate frequency of each topic in the dataset
topic_counts = df['topic'].value_counts()
print("\nTopic counts:\n", topic_counts)


Topic counts:
 topic
environment                              135
white-nationalism                        131
gender                                   128
student-debt                             127
sport                                    126
abortion                                 126
gun-control                              124
immigration                              122
vaccines                                 122
coronavirus                              122
trump-presidency                         120
middle-class                             118
elections-2020                           111
international-politics-and-world-news     88
Name: count, dtype: int64


In [7]:
print(df['Label_bias'].value_counts(normalize=True))
print(df['Label_opinion'].value_counts(normalize=True))

Label_bias
Biased          0.598824
Non-biased      0.313529
No agreement    0.087647
Name: proportion, dtype: float64
Label_opinion
Entirely factual                         0.336471
Expresses writer’s opinion               0.306471
Somewhat factual but also opinionated    0.254706
No agreement                             0.102353
Name: proportion, dtype: float64


In [8]:
# Load additional CSVs for media reliability scores & factuality
factuality = pd.read_csv('data/news_media_factuality.csv')
reliability = pd.read_csv('data/news_media_reliability.csv')


In [9]:
# Mapping the news outlets: outlet name in df --> domain name in the CSVs
outlet_to_domain = {
    'huffpost': 'huffpost.com',
    'msnbc': 'msnbc.com',
    'alternet': 'alternet.org',
    'federalist': 'thefederalist.com',
    'fox-news': 'foxnews.com',
    'breitbart': 'breitbart.com',
    'usa-today': 'usatoday.com',
    'reuters': 'reuters.com'
}

# Create a new column in df for merging
df['merge_domain'] = df['outlet'].map(outlet_to_domain)

# Standardize the domain column in both CSVs for merging (creates 'merge_domain')
factuality['merge_domain'] = factuality['source'].str.lower().str.strip()
reliability['merge_domain'] = reliability['domain'].str.lower().str.strip()

# Merge factuality and reliability using the mapped domain
df = df.merge(factuality[['merge_domain', 'factual_reporting']], on='merge_domain', how='left')
df = df.merge(reliability[['merge_domain', 'reliability_label']], on='merge_domain', how='left')

In [10]:
print(df[['outlet', 'factual_reporting', 'reliability_label']].drop_duplicates())

        outlet factual_reporting  reliability_label
0    usa-today             mixed                  1
1        msnbc             mixed                  1
2     alternet             mixed                  1
3    breitbart             mixed                 -1
4   federalist             mixed                 -1
5      reuters              high                  1
10    huffpost             mixed                  1
19    fox-news             mixed                  1


In [11]:
df.head()

,sentence,news_link,outlet,topic,type,group_id,num_sent,Label_bias,Label_opinion,article,biased_words4,merge_domain,factual_reporting,reliability_label
0,YouTube is making clear there will be no “birt...,https://eu.usatoday.com/story/tech/2020/02/03/...,usa-today,elections-2020,center,1,1,Biased,Somewhat factual but also opinionated,YouTube says no ‘deepfakes’ or ‘birther’ video...,"['belated', 'birtherism']",usatoday.com,mixed,1
1,The increasingly bitter dispute between Americ...,https://www.nbcnews.com/news/sports/women-s-te...,msnbc,sport,left,1,1,Non-biased,Entirely factual,"FRISCO, Texas — The increasingly bitter disput...",['bitter'],msnbc.com,mixed,1
2,So while there may be a humanitarian crisis dr...,https://www.alternet.org/2019/01/here-are-5-of...,alternet,immigration,left,1,1,Biased,Expresses writer’s opinion,Speaking to the country for the first time fro...,['crisis'],alternet.org,mixed,1
3,A professor who teaches climate change classes...,https://www.breitbart.com/politics/2019/05/09/...,breitbart,environment,right,1,1,Non-biased,No agreement,A professor who teaches climate change classes...,['legitimate'],breitbart.com,mixed,-1
4,"Looking around the United States, there is nev...",https://thefederalist.com/2020/03/11/woman-who...,federalist,abortion,right,1,1,Biased,Somewhat factual but also opinionated,The left has a thing for taking babies hostage...,"['killing', 'never', 'developing', 'humans', '...",thefederalist.com,mixed,-1


In [23]:
# export the data frame
df.to_csv('data/new_labeled_dataset.csv', index=False)